### import environment

In [15]:
import gradio as gr
import tempfile
import os
import google.generativeai as genai
import json
from PIL import Image
import time  
import random
import io

### become a lyrics expert

In [16]:
# Initialize the chatbot as a Lyrics Expert
def become_lyrics_expert():
    # Act as a Lyrics Expert
    converted_history.append({"role": "user", "parts": "You are a lyrics expert. Answer only lyrics-related questions."})
    converted_history.append({"role": "model", "parts": "Understood! I’m here to provide insights and suggestions based on song lyrics."})

    # User level
    converted_history.append({"role": "user", "parts": "You're talking to someone who loves analyzing lyrics."})
    converted_history.append({"role": "model", "parts": "Great! I’ll provide detailed analyses and song recommendations based on lyrics."})

    # Keywords in lyrics
    converted_history.append({"role": "user", "parts": "The user is looking for songs containing specific keywords."})
    converted_history.append({"role": "model", "parts": "I’ll suggest songs that contain the provided keywords in their lyrics."})

    # Positive or negative preference
    converted_history.append({"role": "user", "parts": "The user has a preference for songs with positive or negative lyrics."})
    converted_history.append({"role": "model", "parts": "I’ll provide song recommendations based on the user's preference for either positive or negative lyrics."})

    # Sensitive topic (suicide)
    converted_history.append({"role": "user", "parts": "The user may express suicidal thoughts."})
    converted_history.append({"role": "model", "parts": "I’ll respond with positive messages and suggest uplifting songs to provide support and encouragement."})

    # No small talk
    converted_history.append({"role": "user", "parts": "Do not engage in small talk."})
    converted_history.append({"role": "model", "parts": "Understood! I’ll focus solely on lyrics-related inquiries without casual conversation."})

    # Solely Lyrics
    converted_history.append({"role": "user", "parts": "Only discuss lyrics analysis. No way to talk about anything else."})
    converted_history.append({"role": "model", "parts": "I’ll ensure all responses are strictly related to analyzing lyrics, song suggestions, and recommendations."})

# Add to the history
def convert_last_history(converted_history, last_history):
    user_message, model_response = last_history
    if user_message and model_response:
        converted_history.append({"role": "user", "parts": user_message})
        converted_history.append({"role": "model", "parts": model_response})


### function1 Song selection based on keywords

In [17]:
def get_songs_by_keywords(keywords):
    # Mocked song database
    song_database = [
        {"title": "Happy", "artist": "Pharrell Williams", "lyrics": "Because I'm happy..."},
        {"title": "Sad", "artist": "XXX", "lyrics": "I'm feeling so sad..."},
        {"title": "Joy", "artist": "XYZ", "lyrics": "Bring me joy..."}
    ]
    # Filter songs containing the keyword in the lyrics
    matched_songs = [song for song in song_database if any(keyword.lower() in song["lyrics"].lower() for keyword in keywords)]
    return matched_songs

###  Function 2: Song selection based on mood preference


In [18]:
def get_songs_by_mood(preference):
    # Mocked song database with positive/negative sentiments
    song_database = [
        {"title": "Happy", "artist": "Pharrell Williams", "mood": "positive", "lyrics": "Because I'm happy..."},
        {"title": "Depressed", "artist": "Sad Band", "mood": "negative", "lyrics": "I feel so low..."},
        {"title": "Joy", "artist": "XYZ", "mood": "positive", "lyrics": "Bring me joy..."}
    ]
    # Filter songs by mood preference
    matched_songs = [song for song in song_database if song["mood"] == preference]
    return matched_songs


### Function 3: Sensitive topic detection and positive response


In [19]:
def respond_to_sensitive_topics(user_message):
    if "suicide" in user_message.lower():
        positive_response = {
            "response": "I'm really sorry you're feeling this way, but I'm here to support you. Here's a positive song that might lift your spirits.",
            "suggested_songs": get_songs_by_mood("positive")
        }
        return positive_response
    return None

### example usage

In [20]:
def handle_user_request(user_message):
    if "keyword" in user_message:
        keywords = user_message.split()  # Example of extracting keywords from user input
        songs = get_songs_by_keywords(keywords)
        return songs
    elif "positive" in user_message or "negative" in user_message:
        preference = "positive" if "positive" in user_message else "negative"
        songs = get_songs_by_mood(preference)
        return songs
    else:
        sensitive_response = respond_to_sensitive_topics(user_message)
        if sensitive_response:
            return sensitive_response
        return "Let's talk more about the songs you love. Feel free to share your mood or any keywords!"

### image input

### simulate streamed response 

In [21]:
def simulate_typing_response(answer):
    full_sentence = ""
    for word in answer.split():
        full_sentence += word + " "
        yield full_sentence.strip()  
        time.sleep(0.3)  

### rewind function

In [22]:
def rewind_chat(history):
    if len(history) > 0:
        last_safe_state = history[-1]  # Get the last safe state
        print("Rewinding to the last safe state:", last_safe_state)
        return history[:-1]  
    else:
        return history  

In [23]:
def lyrics_expert(text_input, image_input=None, conversation_history=[], temperature=0.7, top_k=50, top_p=0.9):
    response_text = None
    global lyrics_info
    model_config = {
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "max_output_tokens": 4096,
    }

    try:
        # If image is uploaded (potentially for album cover analysis in the future)
        if image_input:  
            # Placeholder for image processing, like analyzing an album cover (extend functionality as needed)
            lyrics_info = identify_album_image(image_input)

            if lyrics_info:
                response_text = f"Here is the JSON description of the uploaded image:\n{lyrics_info}"
            else:
                response_text = "It seems that the album image could not be described. Could you try uploading another one?"
        elif text_input:  # Handle text-based input for lyrics analysis
            # Create a generative model for text-based conversation
            model = genai.GenerativeModel("gemini-1.5-flash", model_config)
            chat = model.start_chat(history=conversation_history)

            # Functionality 1: Responding based on keywords
            if "keyword" in text_input.lower():
                keywords = text_input.split()  # Example of extracting keywords
                response_text = handle_lyrics_keywords(keywords)

            # Functionality 2: Responding based on mood (positive or negative)
            elif "positive" in text_input.lower() or "negative" in text_input.lower():
                mood = "positive" if "positive" in text_input.lower() else "negative"
                response_text = handle_lyrics_mood(mood)

            # Functionality 3: Detecting sensitive topics (like suicidal thoughts) and providing supportive feedback
            elif "suicide" in text_input.lower() or "depressed" in text_input.lower():
                response_text = handle_sensitive_topics(text_input)

            else:
                # General conversation about lyrics or songs
                response = chat.send_message(text_input)
                response_text = response.text if hasattr(response, 'text') else "No response text available."

        else:
            response_text = "Please provide text input or upload an image."

        # Simulate typing response for a more interactive feel
        simulated_responses = list(simulate_typing_response(response_text))

        # Update conversation history
        if text_input:
            conversation_history.append([text_input, None])  # Append user query
        conversation_history.append([None, simulated_responses[-1]])  # Append bot response

    except StopCandidateException as safety_error:
        print("Safety filter triggered. Unable to respond to this query.")
        response_text = "Sorry, your query triggered a safety filter, and I cannot generate a response."
        conversation_history.append([None, response_text])  # Add safety response to history

    return conversation_history, simulated_responses[-1]


# Helper functions

# Function 1: Handle keywords in lyrics
def handle_lyrics_keywords(keywords):
    # Mock database or API integration to search songs by keywords
    song_database = [
        {"title": "Happy", "artist": "Pharrell Williams", "lyrics": "Because I'm happy..."},
        {"title": "Sad", "artist": "XXXTentacion", "lyrics": "I'm feeling so sad..."},
        {"title": "Joy", "artist": "XYZ", "lyrics": "Bring me joy..."}
    ]
    # Filter songs containing the keyword in the lyrics
    matched_songs = [song for song in song_database if any(keyword.lower() in song["lyrics"].lower() for keyword in keywords)]
    if matched_songs:
        return f"Songs matching your keywords: {[song['title'] for song in matched_songs]}"
    else:
        return "No songs found matching your keywords."

# Function 2: Handle mood-based song recommendations
def handle_lyrics_mood(mood):
    # Mock database for mood-based songs
    song_database = [
        {"title": "Happy", "artist": "Pharrell Williams", "mood": "positive"},
        {"title": "Sad", "artist": "XXXTentacion", "mood": "negative"},
        {"title": "Joy", "artist": "XYZ", "mood": "positive"},
    ]
    # Return songs based on the mood (positive/negative)
    matched_songs = [song for song in song_database if song["mood"] == mood]
    if matched_songs:
        return f"Here are some {mood} songs: {[song['title'] for song in matched_songs]}"
    else:
        return f"No {mood} songs found."

# Function 3: Handle sensitive topics (e.g., suicidal thoughts)
def handle_sensitive_topics(user_message):
    # Positive response and uplifting song recommendations
    positive_response = {
        "response": "I'm really sorry you're feeling this way, but I'm here to support you. Here are some positive songs that might lift your spirits.",
        "suggested_songs": handle_lyrics_mood("positive")
    }
    return f"{positive_response['response']} {positive_response['suggested_songs']}"

# Simulate typing response (for interactive feel)
def simulate_typing_response(text):
    # Split text into simulated chunks (optional)
    for i in range(0, len(text), 10):  # Simulate typing in chunks
        yield text[:i + 10]

### interface


In [24]:
import gradio as gr
import gradio.themes  # Import Gradio themes

# Define a theme for the app
theme = gr.themes.Soft(primary_hue="blue", font="Comic Sans MS")

# Example inputs for users to try
examples = [
    ["Can you suggest a song with the keyword 'love'?", None],  # Text-only example
    ["I'm feeling down, can you suggest some positive songs?", None],  # Text with mood
]

# Define the Gradio interface
with gr.Blocks(theme=theme) as app:
    lyrics_chatbot = gr.Chatbot(label="🎶 Your Personal Lyrics Expert 🎶")

    with gr.Row():
        with gr.Column(scale=7):
            user_input = gr.Textbox(label="Ask a Question", placeholder="Ask about song lyrics, moods, or keywords.")
        with gr.Column(scale=3):
            image_input = gr.Image(label="Upload Album Cover (Optional)", type="pil")

    # Add sliders for temperature, top-k, and top-p (Model Parameters)
    temperature_slider = gr.Slider(minimum=0.0, maximum=1.0, value=0.7, label="Temperature")
    top_k_slider = gr.Slider(minimum=0, maximum=100, value=50, label="Top-K")
    top_p_slider = gr.Slider(minimum=0.0, maximum=1.0, value=0.9, label="Top-P")

    submit_btn = gr.Button("Submit")
    rewind_btn = gr.Button("Rewind Chat")

    # Keep the chat history
    conversation_history = gr.State([])

    # Define example inputs (text or image) for users to try
    gr.Examples(
        examples=examples,  
        inputs=[user_input, image_input],  
        label="Try some example questions and requests"
    )

    # Submit button to handle the lyrics expert function
    submit_btn.click(
        fn=lyrics_expert,  # The function handling lyrics queries
        inputs=[user_input, image_input, conversation_history, temperature_slider, top_k_slider, top_p_slider], 
        outputs=[lyrics_chatbot, conversation_history]
    )

    # Rewind button to clear chat history (or go back a step)
    rewind_btn.click(
        fn=lambda conversation_history: ([], "Chat history cleared!"),  # Clear history
        inputs=conversation_history, 
        outputs=[lyrics_chatbot, conversation_history]
    )

# Launch the app
app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/var/folders/31/pnynpc9d6g141fp9hhncsc700000gn/T/ipykernel_63383/2264098060.py", line 23, in lyrics_expert
    model = genai.GenerativeModel("gemini-1.5-flash", model_config)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/chengu/Desktop/anaconda3/lib/python3.11/site-packages/google/generativeai/generative_models.py", line 85, in __init__
    self._safety_settings = safety_types.to_easy_safety_dict(safety_settings)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/chengu/Desktop/anaconda3/lib/python3.11/site-packages/google/generativeai/types/safety_types.py", line 224, in to_easy_safety_dict
    return {to_harm_category(key): to_block_threshold(value) for key, value in settings.items()}
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/chengu/Desktop/anaconda3/lib/python3.11/site-packages/google/generativ